In [ ]:
import sys
import pandas as pd
import numpy as np

In [ ]:
df_olympic = pd.read_csv('/content/Olympic_Athlete_Biography.csv')
df_olympic

In [ ]:
df_result = pd.read_csv('/content/Olympic_Athlete_Event_Details.csv')
df_result

Merge Data by Ahtlete ID

In [ ]:
df_inner = pd.merge(df_result, df_olympic, left_on=['athlete_id'], right_on=['athlete_id'])
df_inner

In [ ]:
print(df_inner.isnull().sum())

นำเอา Event ที่มีชื่อว่า Boys เนื่องจากไม่มีการมอบเหรียญและไม่มีการบันทึกน้ำหนักส่วนสูง

In [ ]:
count_boys = df_inner['event'].str.lower().str.contains('boys', na=False).sum()
print(f"Number of rows where 'event' contains 'boys': {count_boys}")

เปลี่ยนข้อมูลในช่อง Edition ให้เป็นปีที่แข่งทั้งหมดเพื่อนำไปคำนวนอายุของนักกีฬา ณ ปีที่แข่ง

In [ ]:
def remove_non_numeric(text):
  import re
  return re.sub(r'\D', '', text)

df_inner['edition'] = df_inner['edition'].apply(remove_non_numeric)

print(df_inner)

คำนวนและสร้าง Column ใหม่เพื่อเก็บค่าอายุ

In [ ]:
df_inner['edition'] = pd.to_numeric(df_inner['edition'], errors='coerce')

df_inner['born'] = pd.to_datetime(df_inner['born'], errors='coerce')

df_inner['age'] = df_inner.apply(lambda row: row['edition'] - row['born'].year if pd.notnull(row['born']) and pd.notnull(row['edition']) else None, axis=1)

df_inner[['athlete', 'born', 'edition', 'age']].head()

ฟังก์ชันเพื่อแทนที่ข้อมูลน้ำหนักและส่วนสูงที่มีค่าเป็นช่วง ด้วยค่าเฉลี่ย เช่น กรณี '63-67' แทนด้วย (63+67)/2

In [ ]:
def convert_range_to_mean(value):
    if isinstance(value, str) and '-' in value:
        try:
            low, high = map(float, value.split('-'))
            return (low + high) / 2
        except:
            return np.nan
    try:
        return float(value)
    except:
        return np.nan

df_inner['height'] = df_inner['height'].apply(convert_range_to_mean)
df_inner['weight'] = df_inner['weight'].apply(convert_range_to_mean)


แบ่งข้อมูลตามประเภทกีฬาและเพศ คำนวณค่า Mean ของ height, weight, age ในแต่ละกลุ่ม เพื่อเติม Missing Value

In [ ]:
groups = df_inner.groupby(['sport', 'sex'])

mean_values = groups[['height', 'weight', 'age']].mean()

def fill_missing_values(row):
    if pd.isnull(row['height']):
        row['height'] = mean_values.loc[(row['sport'], row['sex']), 'height']
    if pd.isnull(row['weight']):
        row['weight'] = mean_values.loc[(row['sport'], row['sex']), 'weight']
    if pd.isnull(row['age']):
        row['age'] = mean_values.loc[(row['sport'], row['sex']), 'age']
    return row

df_inner = df_inner.apply(fill_missing_values, axis=1)

df_inner[['sport', 'sex', 'height', 'weight', 'age']].head()

In [ ]:
df_inner.dropna(subset=['height', 'weight', 'age'], inplace=True)

print(df_inner.isnull().sum())

ลบข้อมูลใน Column pos ที่ไม่ได้เป็นตัวเลขและตัวเลขที่ไม่แสดงอันดับของการแข่งชัน

In [ ]:
import re

def is_numeric_ranking(pos):
  """
  Checks if the ranking is numeric.
  It should not contain 'h', 'r' or '/'.
  """
  match = re.match(r'^\d+$', str(pos))
  return bool(match)

df_inner_cleaned = df_inner[df_inner['pos'].apply(is_numeric_ranking)]

df_inner_cleaned[['athlete', 'pos']].head()

In [ ]:
df_inner_cleaned.to_csv('df_inner_cleaned.csv', index=False)

In [ ]:
import altair as alt
df_medals_by_country = df_inner_cleaned.groupby('country')['medal'].count().reset_index(name='medal_count')

In [ ]:
df_medals_by_country = df_inner_cleaned.groupby('country')['medal'].count().reset_index(name='medal_count').sort_values(by='medal_count', ascending=False)

alt.Chart(df_medals_by_country).mark_bar().encode(
    x='medal_count',
    y=alt.Y('country', sort='-x')
)

In [ ]:
df_athletes_by_sport = df_inner_cleaned.groupby('sport')['athlete'].nunique().reset_index(name='athlete_count').sort_values(by='athlete_count', ascending=False)

bars = alt.Chart(df_athletes_by_sport).mark_bar().encode(
    x='athlete_count',
    y=alt.Y('sport', sort='-x')
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # distance from bar to text
).encode(
    text='athlete_count'
)

(bars + text)

In [ ]:
sports = ['Athletics', 'Swimming', 'Rowing', 'Football', 'Sailing', 'Ice Hockey', 'Shooting', 'Artistic Gymnastics'] # Added 'Artistic Gymnastics'
avg_height = []

for sport in sports:
  avg_height_medal_winners = df_inner_cleaned[(df_inner_cleaned['medal'].notnull()) & (df_inner_cleaned['sport'] == sport)]['height'].mean()
  avg_height_others = df_inner_cleaned[(df_inner_cleaned['medal'].isnull()) & (df_inner_cleaned['sport'] == sport)]['height'].mean()
  avg_height.append([sport, 'Medal Winners', avg_height_medal_winners])
  avg_height.append([sport, 'Others', avg_height_others])

df_avg_height = pd.DataFrame(avg_height, columns=['sport', 'athlete_type', 'avg_height'])

alt.Chart(df_avg_height).mark_bar().encode(
    x='athlete_type',
    y='avg_height',
    color='athlete_type',
    column='sport'
)

In [ ]:
sports = ['Athletics', 'Swimming', 'Rowing', 'Football', 'Sailing', 'Ice Hockey', 'Shooting', 'Artistic Gymnastics']
avg_weight = []

for sport in sports:
  avg_weight_medal_winners = df_inner_cleaned[(df_inner_cleaned['medal'].notnull()) & (df_inner_cleaned['sport'] == sport)]['weight'].mean()
  avg_weight_others = df_inner_cleaned[(df_inner_cleaned['medal'].isnull()) & (df_inner_cleaned['sport'] == sport)]['weight'].mean()
  diff = avg_weight_medal_winners - avg_weight_others
  avg_weight.append([sport, 'Medal', avg_weight_medal_winners, diff]) # Changed 'Medal Winners' to 'Medal'
  avg_weight.append([sport, 'No Medal', avg_weight_others, diff]) # Changed 'Others' to 'No Medal'

df_avg_weight = pd.DataFrame(avg_weight, columns=['sport', 'athlete_type', 'avg_weight', 'diff'])

# Create the base chart with layering first
bars = alt.Chart(df_avg_weight).mark_bar().encode(
    x='athlete_type',
    y='avg_weight',
    color='athlete_type'
)

# Apply faceting to the layered chart
(bars).facet(
    column='sport'
)

In [ ]:
sports = ['Athletics', 'Swimming', 'Rowing', 'Football', 'Sailing', 'Ice Hockey', 'Shooting', 'Artistic Gymnastics'] # Added 'Artistic Gymnastics'
age = []

for sport in sports:
  age_medal_winners = df_inner_cleaned[(df_inner_cleaned['medal'].notnull()) & (df_inner_cleaned['sport'] == sport)]['age'].mean()
  age_others = df_inner_cleaned[(df_inner_cleaned['medal'].isnull()) & (df_inner_cleaned['sport'] == sport)]['age'].mean()
  diff = age_medal_winners - age_others
  age.append([sport, 'Medal Winners', age_medal_winners, diff])
  age.append([sport, 'Others', age_others, diff])

df_age = pd.DataFrame(age, columns=['sport', 'athlete_type', 'avg_age', 'diff'])

# Create the base chart with layering first
bars = alt.Chart(df_age).mark_bar().encode(
    x='athlete_type',
    y='avg_age',
    color='athlete_type'
)

# Apply faceting to the layered chart
(bars).facet(
    column='sport'
)

In [ ]:
import altair as alt
import pandas as pd

alt.data_transformers.disable_max_rows() # Disable the max rows limit

# Create a new column 'medal_won' based on 'medal' column
df_inner_cleaned['medal_won'] = df_inner_cleaned['medal'].apply(lambda x: 'Medal' if not pd.isnull(x) else 'No Medal')

# Height vs Weight Chart with color gradient for age
alt.Chart(df_inner_cleaned).mark_point().encode(
    x='height',
    y='weight',
    color=alt.Color('age', scale=alt.Scale(scheme='viridis')),  # Color gradient for age
    size='age'  # Adjust point size based on age
).properties(
    title='Height vs Weight for Medal Winners and Others'
).facet(
    row='medal_won'  # Separate the chart by medal_won into rows
)

In [ ]:
df_inner_cleaned['pos'] = pd.to_numeric(df_inner_cleaned['pos'])

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate the correlation matrix
corr = df_inner_cleaned[['height', 'weight', 'age', 'pos']].corr()

# Create a heatmap using seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define a list of sports to include in the correlation matrices
sports = ['Athletics', 'Swimming', 'Rowing', 'Football', 'Sailing', 'Ice Hockey', 'Shooting', 'Artistic Gymnastics']

# Create a figure with subplots for each sport
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(20, 10))
axes = axes.flatten()

# Iterate over the sports and create a correlation matrix for each one
for i, sport in enumerate(sports):
    # Filter the DataFrame for the current sport
    df_sport = df_inner_cleaned[df_inner_cleaned['sport'] == sport]

    # Calculate the correlation matrix
    corr = df_sport[['height', 'weight', 'age', 'pos']].corr()

    # Create a heatmap for the current sport
    sns.heatmap(corr, annot=True, cmap='coolwarm', ax=axes[i])
    axes[i].set_title(f'Correlation Matrix for {sport}')

# Adjust the layout and display the plot
plt.tight_layout()
plt.show()